In [1]:
from alfabeto_sources.all_sources import *
from alfabeto_code.AlfabetoConverter import transposed_pc_chords_noMMD
from alfabeto_data.pickled_data import *
from alfabeto_data.harmonic_function_data import *
from Continuo.ContinuoConverter import figure_intervals_pc, transposed_figures
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import write_dot
from decimal import Decimal

import csv, pickle

In [2]:
def bigram_numbers_alfabeto(corpus, modes):
    X = []
    csv_data = []
    for book in corpus:
        for song in book.values():
    #         print(song['alfabeto'])
            attempt_mode = (song['data']['key'], song['data']['final'])
            if modes == 'all':
                X.append(transposed_pc_chords_noMMD(song['alfabeto'], 'all'))
            elif attempt_mode in modes:
                X.append(transposed_pc_chords_noMMD(song['alfabeto'], 'all'))
    numerals = ['I', 'i', 'bII', 'bii', 'II', 'ii',
            'bIII', 'biii', 'III', 'iii', 'IV', 'iv',
            '#IV', '#iv', 'V', 'v', 'bVI', 'bvi',
            'VI', 'vi', 'bVII', 'bvii', 'VII', 'vii']
    numeral_data = {}
    final_numerals = []
    total_chords = 0
    for x in numerals:
        numeral_data[x] = 0
    for x in X:
        for y in x:
            numeral_data[y] += 1
            total_chords += 1
    for x in numerals:
        if numeral_data[x] >= .01*total_chords:
            final_numerals.append(x)
    after_labels = [' ']
    for x in final_numerals:
        after_labels.append(x)
    csv_data.append(after_labels)
    for numeral in final_numerals:
        csv_temp = []
        csv_temp.append(numeral)
        numeral_after_dict = {}
        total_numbers_after = 0
        for x in numerals:
            numeral_after_dict[x] = 0
        for x in X:
            for y in range(1, len(x)):
                if x[y-1] == numeral:
                    numeral_after_dict[x[y]] += 1
                    total_chords += 1
        numeral_after = []
        for x in final_numerals:
            csv_temp.append(numeral_after_dict[x])
        csv_data.append(csv_temp)    

    print(total_chords)
    return csv_data

def bigram_numbers_continuo(alfabeto_source, modes):
    csv_data = []
    all_keys = []
    X = []
    for book in alfabeto_source:
        for song in book.values():
    #         print(song['alfabeto'])
            attempt_mode = (song['data']['key'], song['data']['final'])
#             print(attempt_mode)
            if modes == 'all':
                X.append(figure_intervals_pc(song, 'all'))
            elif attempt_mode in modes:
                X.append(figure_intervals_pc(song, 'all'))
    all_chords = {}
    all_chord_numbers = 0
    for x in X:
        for y in x:
            all_chord_numbers += 1
    for x in X:
        for y in x:
            if str(y) not in all_chords.keys():
                all_chords[str(y)] = 1
            else:
                all_chords[str(y)] += 1
    all_chords_final = []
    for x, y in all_chords.items():
        if y/all_chord_numbers >= .01:
            all_chords_final.append(x)
    
    csv_data = []
    after_labels = [' ']
    for x in all_chords_final:
        after_labels.append(x)
    csv_data.append(after_labels)
    total_chords = 0
    for numeral in all_chords_final:
        csv_temp = []
        csv_temp.append(numeral)
        numeral_after_dict = {}
        total_numbers_after = 0
        for x in all_chords_final:
            numeral_after_dict[x] = 0
        for x in X:
            for y in range(1, len(x)):
                if str(x[y-1]) == numeral and str(x[y]) in all_chords_final and str(x[y-1]) != str(x[y]):
                    numeral_after_dict[str(x[y])] += 1
                    total_chords += 1
        numeral_after = []
        for x in all_chords_final:
            csv_temp.append(numeral_after_dict[x])
        csv_data.append(csv_temp)   
    return csv_data

def bigram_numbers_corpus(corpus_data_tuples, modes): #chord/key data into csv-ready functions
    csv_data = []
    all_keys = []
    X = []
    for x, y in zip(corpus_data_tuples[0], corpus_data_tuples[1]):
        if modes == 'all':
            temp_list = []
            for z in x:
                temp_list.append(''.join(map(str, z)))
            X.append(temp_list)
            print(temp_list)
            all_keys.append(y)
        elif y in modes:
            temp_list = []
            for z in x:
                temp_list.append(''.join(map(str, z)))
            X.append(temp_list)
#             print(temp_list)
            all_keys.append(y)
    all_chords = {}
    all_chord_numbers = 0
    for x in X:
        for y in x:
            all_chord_numbers += 1
    for x in X:
        for y in x:
            if str(y) not in all_chords.keys():
                all_chords[str(y)] = 1
            else:
                all_chords[str(y)] += 1
    all_chords_final = []
    for x, y in all_chords.items():
        if y/all_chord_numbers >= .01:
            all_chords_final.append(x)
    
    csv_data = []
    after_labels = [' ']
    for x in all_chords_final:
        after_labels.append(x)
    csv_data.append(after_labels)
    total_chords = 0
    for numeral in all_chords_final:
        csv_temp = []
        csv_temp.append(numeral)
        numeral_after_dict = {}
        total_numbers_after = 0
        for x in all_chords_final:
            numeral_after_dict[x] = 0
        for x in X:
            for y in range(1, len(x)):
                if str(x[y-1]) == numeral and str(x[y]) in all_chords_final and str(x[y-1]) != str(x[y]):
                    numeral_after_dict[str(x[y])] += 1
                    total_chords += 1
        numeral_after = []
        for x in all_chords_final:
            csv_temp.append(numeral_after_dict[x])
        csv_data.append(csv_temp)    

    print(total_chords)
    return csv_data

In [3]:
def song_chords(chord_data): #converts chordify/key data into chord/key data
#     chord_data = []
    final_data = []
#     for x in chordified_song:
#         bass_note = chordified_song[-1].bass().pitchClass
#         chord_data.append(sorted(set([(y.pitchClass - bass_note)%12 for y in x])))
    for x in range(1, len(chord_data)):
        if chord_data[x-1] != chord_data[x] and len(chord_data[x-1][-1]) >= 3:
            final_data.append(chord_data[x-1])
#         elif len(final_data) > 0 and final_data[-1] == chord_data[x-1]:
#             final_data[x-1] = chord_data[x-1]
#         elif len(chord_data[x-1][-1]) >= 3:
            
#         elif chord_data[x-1] == chord_data[x]:
# #             if set(chord_data[x-1][-1]).issubset(chord_data[x][-1]):
# #                 print(set(chord_data[x-1][-1]), chord_data[x][-1])
#             if len(final_data) > 0 and final_data[-1] == chord_data[x-1]:
#                 print(final_data)
#                 final_data[x-1] = chord_data[x]
#             elif len(chord_data[x][-1]) >= 3:
#                 final_data.append(chord_data[x])
#             elif len(chord_data[x-1][-1]) >= 3:
#                 if len(final_data) > 0 and chord_data[x-1] != final_data[-1]:
#                     final_data.append(chord_data[x-1])
#             elif set(chord_data[x-1][-1]).issuperset(chord_data[x][-1]) is False and len(chord_data[x][-1]) >= 3:
#                 final_data.append(chord_data[x])
    return final_data

major = [('f', 5), ('n', 0), ('f', 0), ('n', 7), ('f', 10), ('n', 5), 
         (-1, 5), (0, 0), (-1, 0), (0, 7), (-1, 10), (0, 5) ]
minor= [('n', 2), ('n', 9), ('f', 2), ('f', 7), ('n', 4), 
        (0, 2), (0, 9), (-1, 2), (-1, 7), (0, 4)]
# be = bigram_numbers(GetAll.all_alf, 'all')
tonal_major = {(0,0): 'CM', (1, 7): 'GM',(2, 2): 'DM', (3, 9): 'AM', (4, 4): 'EM', (5, 11): 'BM', (6, 6): 'F♯M',
               (-1,5): 'FM', (-2,10): 'B♭M', (-3,3): 'E♭M', (-4,8): 'A♭M', (-5,1): 'D♭M', (-6,6): 'G♭M'}
tonal_minor = {(0,9): 'am', (1, 4): 'em', (2, 11): 'bm', (3, 6): 'f♯m', (4, 1): 'c♯m', (5, 8): 'g♯m', (6, 3): 'd♯m', 
               (-1,2): 'dm', (-2,7): 'gm', (-3,0): 'cm', (-4,5): 'fm', (-5,10): 'b♭m', (-6,3): 'e♭m'}
bach_major = [x for x in tonal_major.keys()]
bach_minor = [x for x in tonal_minor.keys()]

In [6]:
# pal_csv = bigram_numbers_corpus(bach_chords_data, bach_minor)
alf_continuo_csv = bigram_numbers_continuo(GetAll.all_alf, minor)
# song_chords(bach_continuo_data[0][0])
# for x in GetAll.all_alf:
#     for y in x.values():
# #         print(y)
#         print(transposed_figures(y))

In [32]:
alf_co

True

In [5]:
CSV_DATA = alf_continuo_csv

NameError: name 'alf_continuo_csv' is not defined

In [ ]:
with open('/home/daniel/Desktop/alf_continuo_minor.csv', 'w', newline='') as fp:
    a = csv.writer(fp, delimiter=',')
    final_data = CSV_DATA
    a.writerows(final_data)

In [ ]:
x = 0.23543434
y = Decimal(str(x),).quantize(Decimal('.00'))
print(y)

In [3]:
def function_mapper(corpus, modes, cluster, source, path):
    X = []
    csv_data = []
    if source == 'alfabeto':
        for book in corpus:
            for song in book.values():
        #         print(song['alfabeto'])
                attempt_mode = (song['data']['key'], song['data']['final'])
                if modes == 'all':
                    X.append(transposed_pc_chords_noMMD(song['alfabeto'], 'all'))
                elif attempt_mode in modes:
                    X.append(transposed_pc_chords_noMMD(song['alfabeto'], 'all'))
    elif source == 'alfabeto continuo':
        for book in corpus:
            for song in book.values():
                attempt_mode = (song['data']['key'], song['data']['final'])
                if modes == 'all':
                    X.append(figure_intervals_pc(song, 'all'))
                elif attempt_mode in modes:
                    X.append(figure_intervals_pc(song, 'all'))
    else:
        for x, y in zip(corpus[0], corpus[1]):
            if modes == 'all':
                X.append(x)
            elif y in modes:
                X.append(x)
#                 print(song_chords(x))
    function_data = []
    for x in X:
        temp_list = []
        for y in x:
            for a, b in cluster.items():
                if y in b:
                    temp_list.append(a)
        function_data.append(temp_list)
    cluster_dict = {}
    for x in cluster.keys():
        for y in cluster.keys():
            cluster_dict[(x, y)] = 0
    for x in function_data:
        for y in range(1, len(x)):
            cluster_dict[(x[y-1], x[y])] += 1
    total_chord_changes = 0
    for x in cluster_dict.values():
        total_chord_changes += x
    print('total chord changes:', total_chord_changes)
    
    #networkx graph
    G = nx.DiGraph()
    for x, y in cluster_dict.items():
        if y > .01*total_chord_changes:
            G.add_edge(x[0], x[1], penwidth=y/total_chord_changes * 30) #
#                        label=Decimal(str(y/total_chord_changes*100)).quantize(Decimal('.00')))
    H = nx.Graph() #make 
    
    edge_weights = []
    edges = G.edges(data=True)
    print('edges:', edges)
    for x in edges:
        edge_weights.append(x[2]['penwidth'])
    pos = nx.circular_layout(G)
    nx.draw_networkx_edges(G, pos, edges=edges, edge_color='green', width=edge_weights, alpha=.5)
    nx.draw_networkx_nodes(G, pos, node_color='green')
    nx.draw_networkx_labels(G, pos)
    write_dot(G, '/home/daniel/Desktop/pal_minor_6.dot')
    print(cluster_dict)



In [ ]:
# function_mapper(palestrina_chords_data, minor, palestrina_minor[6], 'other')
function_mapper(GetAll.all_alf, major, alfabeto_continuo_major[3], 'alfabeto continuo')

In [ ]:
def song_function_analysis(song, cluster_data):
    song_numerals = transposed_pc_chords_noMMD(song, 'all')
    function_labels = []
    for numeral in song_numerals:
        for labels, clusters in cluster_data.items():
            if numeral in clusters:
                function_labels.append(labels)
    print(song_numerals)
    print(function_labels)
    print(len(song_numerals), len(function_labels))

In [ ]:
A = GetComposer.all_obizzi[0][1]['alfabeto']
print(GetComposer.all_obizzi[0][1]['data'])

In [ ]:
song_function_analysis(A, alfabeto_major[3])

In [ ]:
from Continuo.ContinuoMarkov import markovMaker

In [ ]:
mark = markovMaker(GetAll.all_alf, 'all', 2, 'joined')

In [ ]:
len(mark)

In [ ]:
all_numbers = 0
for a, b in mark.items():
    for c in b.values():
        all_numbers += c

reduced_dict = {}
for a, b in mark.items():
    for c, d in b.items():
        if d/all_numbers >= .01:
            reduced_dict[a] = {c:d}
print(all_numbers)

In [ ]:
print(len(mark))
print(len(reduced_dict))

In [ ]:
figure_intervals_pc(song, mode):